# Analysis of Data Job Market in Latvia
## Data Collection and preparation

### 0.Setting up Virtual Environment

In [1]:
!pip install virtualenv
!virtualenv myenv

created virtual environment CPython3.9.7.final.0-64 in 848ms
  creator CPython3Windows(dest=C:\Users\ernes\OneDrive\Mokslai\SGT- Final Project\Data_Job_Market_Analysis_LV\myenv, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=C:\Users\ernes\AppData\Local\pypa\virtualenv)
    added seed packages: pip==23.1.2, setuptools==67.7.2, wheel==0.40.0
  activators BashActivator,BatchActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [2]:
!myenv\Scripts\activate.bat

### 1. Data Collection

In [3]:
import requests
from bs4 import BeautifulSoup

In [4]:
import pandas as pd

In [5]:
import re

### Latvian cv.lv

In [6]:
url = 'https://www.cv.lv/lv/search?limit=2000&offset=20&keywords%5B0%5D=data%20analyst&keywords%5B1%5D=data%20scientist&keywords%5B2%5D=data%20engineer&keywords%5B3%5D=machine%20learning&keywords%5B4%5D=analyst&keywords%5B5%5D=anal%C4%ABti%C4%B7is&keywords%5B6%5D=datu&keywords%5B7%5D=datu%20zin%C4%81tnieks&keywords%5B8%5D=big%20data%20analytics&sorting=EXPIRING&fuzzy=false&suitableForRefugees=false&isHourlySalary=false&isRemoteWork=false&isQuickApply=false'
response = requests.get(url)

In [7]:
soup = BeautifulSoup(response.content, 'html.parser')

In [8]:
vacancy_items = soup.find_all("li", class_="jsx-1871295890 jsx-2661613696 vacancies-list__item false")

In [9]:
vacancies = []
for item in vacancy_items:
    title = item.find("span", class_="jsx-1401030249 vacancy-item__title").text.strip()
    vacancy_item = item.find('a', {'class': 'vacancy-item'})
    vacancy_url = vacancy_item.get('href')
    end_date = item.find("span", class_="jsx-1401030249 vacancy-item__expiry").text.strip()
                                
    location = item.find("span", class_="jsx-1401030249 vacancy-item__locations").text.strip()
    salary = item.find("span", class_="jsx-1401030249 vacancy-item__salary-label")
    if salary is not None:
        salary = salary.text.strip()
    
    info_secondary = item.find('div', {'class': 'vacancy-item__info-secondary'})
    info_secondary_text = info_secondary.get_text()
    
    time_patterns = '(dien(a|s)|stund(a|s)|mēne(ši|sis)|sekund(e|es)|minūt(es|e))'
    pattern = f'Publicēts pirms (\d+) {time_patterns}|Atjaunināts pirms (\d+) {time_patterns}'
    match = re.search(pattern, info_secondary_text)
    published = match.group(0)
    
    p=1
    for a_tag in item.find_all('a', {'class': 'jsx-1401030249'}):
        p=p+1
        if p==3:
            company_name = a_tag.text
            company_url = a_tag['href']
        
      
    vacancies.append({
        "Title": title,
        "Vacancy URL" : vacancy_url,
        "Company name": company_name,
        "Company URL" :company_url,
        "Location": location,
        "Published": published,
        "End_date" : end_date, 
        "Salary": salary
    })

df = pd.DataFrame(vacancies)

### Lithuanian cv.lt

In [10]:
vacancies_lt = []

for page in range(1, 10): # Change the range according to the number of pages you want to scrape
    url_lt = f"https://www.cv.lt/darbo-paieska/data?page={page}&texts=data"
    response = requests.get(url_lt)
    soup = BeautifulSoup(response.content, "html.parser")
    vacancy_items = soup.select(".job-wr")

    for item in vacancy_items:
        title = item.find("button", class_="title").text.strip()
        vacancy_item= soup.select_one('.job-wr')
        vacancy_url = vacancy_item['href']
        company_name = item.find("span", {"class": "company"}).find("button").text.strip()

        # for Lithuanian website, company url is not available, leaving for merging purposes
        company_url = "NaN"

        location = item.find("span", {"class": "company"}).find("span").text.strip()
        published = item.select_one(".time span").text

        # for Lithuanian website, expiry date is not available, leaving for merging purposes
        end_date = "NaN"

        salary = item.find("span", {"class": "salary"})
        if salary is not None:
            salary = salary.text.strip()
        else:
            salary='NaN'

        vacancies_lt.append({
                "Title": title,
                "Vacancy URL": vacancy_url,
                "Company name": company_name,
                "Company URL": company_url,
                "Location": location,
                "Published": published,
                "End_date": end_date,
                "Salary": salary,
            })

df_lt = pd.DataFrame(vacancies_lt)

### 3. Data Cleaning

### cv.lv

In [11]:
cv_lv_link = r'https://www.cv.lv'
df['Vacancy URL'] = cv_lv_link + df['Vacancy URL']
df['Company URL'] = cv_lv_link + df['Company URL']

In [12]:
df['End_date'] = df['End_date'].str.replace('Beidzas: ', '')

In [13]:
df['Location'] = df['Location'].str.replace('—', '')
# split Location column into City, Region, Country
splitting = lambda x: pd.Series([i for i in reversed(x.split(','))])
df[['Country', 'Region', 'City']] = df['Location'].apply(splitting)

# Remove the original Location column
df.drop('Location', axis=1, inplace=True)

In [14]:
df

,Title,Vacancy URL,Company name,Company URL,Published,End_date,Salary,Country,Region,City
0,Customer Communication Project Manager,https://www.cv.lv/lv/vacancy/992818/luminor-ba...,Luminor Bank AS Latvijas filiāle,https://www.cv.lv/lv/search/employer/luminor-b...,Publicēts pirms 15 diena,13.05.2023,€ 1940 – 2430,Latvija,Rīgas rajons,Rīga
1,Personāla attīstības speciālists/-e,https://www.cv.lv/lv/vacancy/985020/latvijas-d...,"Latvijas dzelzceļš, VAS",https://www.cv.lv/lv/search/employer/latvijas-...,Atjaunināts pirms 2 diena,13.05.2023,€ 1100 – 1500,Latvija,Rīgas rajons,Rīga
2,GRAUDU PIRMAPSTRĀDES KOMPLEKSA DARBINIEKS-/CE,https://www.cv.lv/lv/vacancy/985019/vaks-koope...,VAKS KOOPERATĪVĀ SABIEDRĪBA,https://www.cv.lv/lv/search/employer/vaks-koop...,Atjaunināts pirms 2 diena,13.05.2023,€ 1300,Latvija,Jelgavas rajons,Jelgava
3,C++ SOFTWARE ENGINEER - UI/UX,https://www.cv.lv/lv/vacancy/984816/ubiquiti-l...,Ubiquiti (Latvia) SIA,https://www.cv.lv/lv/search/employer/ubiquiti-...,Atjaunināts pirms 17 stunda,13.05.2023,€ 3000 – 7000,Latvija,Rīgas rajons,Rīga
4,Informāciju sistēmu testētājs/-a (Middle QA En...,https://www.cv.lv/lv/vacancy/984685/workingday...,WorkingDay Latvia,https://www.cv.lv/lv/search/employer/workingda...,Publicēts pirms 1 mēnesis,13.05.2023,€ 2700 – 3500,Latvija,Rīgas rajons,Rīga
...,...,...,...,...,...,...,...,...,...,...
1001,Test Engineer,https://www.cv.lv/lv/vacancy/993940/visma/test...,VISMA,https://www.cv.lv/lv/search/employer/visma?emp...,Publicēts pirms 11 diena,30.06.2023,€ 2000 – 3000,Latvija,Rīgas rajons,Rīga
1002,Senior Test Engineer,https://www.cv.lv/lv/vacancy/995138/visma/seni...,VISMA,https://www.cv.lv/lv/search/employer/visma?emp...,Publicēts pirms 9 diena,30.06.2023,€ 2400 – 3000,Latvija,Rīgas rajons,Rīga
1003,Security Manager,https://www.cv.lv/lv/vacancy/987624/visma/secu...,VISMA,https://www.cv.lv/lv/search/employer/visma?emp...,Publicēts pirms 25 diena,30.06.2023,€ 3500 – 4500,Latvija,Rīgas rajons,Rīga
1004,Automatizācijas sistēmu inženieris/ Vadošais a...,https://www.cv.lv/lv/vacancy/992319/belam-riga...,Belam - Riga SIA,https://www.cv.lv/lv/search/employer/belam-rig...,Atjaunināts pirms 2 diena,27.07.2023,€ 2000,Latvija,Rīgas rajons,Rīga


In [15]:
# Remove the euro sign
df['Salary'] = df['Salary'].str.replace('€', '').str.strip()


# Check the format of each entry and apply the corresponding operation
df[['Salary Range min', 'Salary Range max']] = df['Salary'].str.split('–', n=1, expand=True)


In [16]:
df['Salary Range min'] = df['Salary Range min'].str.replace('/st.', '').str.strip().astype(float)
df['Salary Range max'] = df['Salary Range max'].str.replace('/st.', '').str.strip().astype(float)


C:\Users\ernes\AppData\Local\Temp/ipykernel_21952/1641069593.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Salary Range min'] = df['Salary Range min'].str.replace('/st.', '').str.strip().astype(float)
C:\Users\ernes\AppData\Local\Temp/ipykernel_21952/1641069593.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Salary Range max'] = df['Salary Range max'].str.replace('/st.', '').str.strip().astype(float)


In [17]:
for i in df.index:
    if pd.notna(df['Salary'][i]) and isinstance(df['Salary'][i], str) and '/st.' in df['Salary'][i]:
        if pd.notna(df['Salary Range min'][i]) and pd.notna(df['Salary Range max'][i]):
            df.loc[i, ['Salary Range min', 'Salary Range max']] *= 21.62 * 8


In [18]:
# Round the values to 2 decimal places
df['Salary Range min'] = df['Salary Range min'].round(2)
df['Salary Range max'] = df['Salary Range max'].round(2)

In [19]:
# remove the original Salary column
df.drop('Salary', axis=1, inplace=True)

In [20]:
import re
from datetime import datetime, timedelta

def get_date_from_string(s):
    if "dien" in s:
        days_ago = int(re.search(r'\d+', s).group())
        return (datetime.now() - timedelta(days=days_ago)).strftime('%d.%m.%Y')
    elif "mēnesi" in s:
        months_ago = int(re.search(r'\d+', s).group())
        return (datetime.now() - timedelta(days=months_ago*30)).strftime('%d.%m.%Y')
    elif "seku" in s or "min" in s or "stund" in s:
        return datetime.now().strftime('%d.%m.%Y')

# Apply the function to the column 'Published'
df['Published'] = df['Published'].apply(get_date_from_string)

### cv.lt

In [21]:
# Adding beginning for vacancy url to show full link
cv_lt_link = r'https://www.cv.lt'
df_lt['Vacancy URL'] = cv_lt_link + df_lt['Vacancy URL']

In [22]:
# split Location field, if more than one city- to show as 'Various'
df_lt['Location'] = df_lt['Location'].apply(lambda x: 'Various' if isinstance(x, str) and ',' in x else x)

In [23]:
# Replace Location field 'Kitas' to 'Other' 
df_lt['Location'] = df_lt['Location'].str.replace('Kitas', 'Other').str.strip()

In [24]:
# Adding columns to match LV formating for location

Country='Lithuania'
df_lt.insert(loc=df_lt.columns.get_loc('End_date') + 2, column='Country', value=Country)
df_lt.insert(loc=df_lt.columns.get_loc('End_date')+3,column='Region', value='Nan')
df_lt.insert(loc=df_lt.columns.get_loc('End_date')+4,column='City', value=df_lt['Location'])


# Remove the original Location column
df_lt.drop('Location', axis=1, inplace=True)

In [26]:
# Cleaning Salary field, if it is showing different value than salary amount, to remove and show it as 0
def clean_salary(salary):
    if salary.startswith("Pareiginės "):
        return 'NaN
    else:
        return salary
    
df_lt["Salary"] = df_lt["Salary"].apply(clean_salary)

In [27]:
# Remove the euro sign 
df_lt['Salary'] = df_lt['Salary'].str.replace('€', '').str.strip()

# Remove the 'Nuo..' word
df_lt['Salary'] = df_lt['Salary'].str.replace('Nuo', '').str.strip()

# Remove the dot as a thousands separator
df_lt['Salary'] = df_lt['Salary'].str.replace('.', '').str.strip()

C:\Users\ernes\AppData\Local\Temp/ipykernel_21952/1120974158.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_lt['Salary'] = df_lt['Salary'].str.replace('.', '').str.strip()


In [28]:
# Spliting Salary field into two columns
df_lt[['Salary Range min', 'Salary Range max']] = df_lt['Salary'].str.extract(r'^(.*?)(?:-(.*))?$')

In [29]:
df_lt['Salary Range min'] = df_lt['Salary Range min'].str.replace('/st.', '').str.strip().astype(float)
df_lt['Salary Range max'] = df_lt['Salary Range max'].str.replace('/st.', '').str.strip().astype(float)

C:\Users\ernes\AppData\Local\Temp/ipykernel_21952/35191948.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_lt['Salary Range min'] = df_lt['Salary Range min'].str.replace('/st.', '').str.strip().astype(float)
C:\Users\ernes\AppData\Local\Temp/ipykernel_21952/35191948.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_lt['Salary Range max'] = df_lt['Salary Range max'].str.replace('/st.', '').str.strip().astype(float)


In [30]:
# Round the values to 2 decimal places
df_lt['Salary Range min'] = df_lt['Salary Range min'].round(2)
df_lt['Salary Range max'] = df_lt['Salary Range max'].round(2)

In [31]:
# remove the original Salary column
df_lt.drop('Salary', axis=1, inplace=True)

In [32]:
# Formating job posting to show date instead of 'Before X days/months' etc.

import re
from datetime import datetime, timedelta

def get_date_from_string(s):
    if "d." in s:
        days_ago = int(re.search(r'\d+', s).group())
        return (datetime.now() - timedelta(days=days_ago)).strftime('%d.%m.%Y')
    elif "mėn." in s:
        months_ago = int(re.search(r'\d+', s).group())
        return (datetime.now() - timedelta(days=months_ago*30)).strftime('%d.%m.%Y')
    elif "val." in s or "min" in s:
        return datetime.now().strftime('%d.%m.%Y')

# Apply the function to the column 'Published'
df_lt['Published'] = df_lt['Published'].apply(get_date_from_string)

In [33]:
# Merging both dataframes into one
full_df = pd.concat([df, df_lt], ignore_index=True)

In [34]:
full_df

,Title,Vacancy URL,Company name,Company URL,Published,End_date,Country,Region,City,Salary Range min,Salary Range max
0,Customer Communication Project Manager,https://www.cv.lv/lv/vacancy/992818/luminor-ba...,Luminor Bank AS Latvijas filiāle,https://www.cv.lv/lv/search/employer/luminor-b...,28.04.2023,13.05.2023,Latvija,Rīgas rajons,Rīga,1940.0,2430.0
1,Personāla attīstības speciālists/-e,https://www.cv.lv/lv/vacancy/985020/latvijas-d...,"Latvijas dzelzceļš, VAS",https://www.cv.lv/lv/search/employer/latvijas-...,11.05.2023,13.05.2023,Latvija,Rīgas rajons,Rīga,1100.0,1500.0
2,GRAUDU PIRMAPSTRĀDES KOMPLEKSA DARBINIEKS-/CE,https://www.cv.lv/lv/vacancy/985019/vaks-koope...,VAKS KOOPERATĪVĀ SABIEDRĪBA,https://www.cv.lv/lv/search/employer/vaks-koop...,11.05.2023,13.05.2023,Latvija,Jelgavas rajons,Jelgava,1300.0,NaN
3,C++ SOFTWARE ENGINEER - UI/UX,https://www.cv.lv/lv/vacancy/984816/ubiquiti-l...,Ubiquiti (Latvia) SIA,https://www.cv.lv/lv/search/employer/ubiquiti-...,13.05.2023,13.05.2023,Latvija,Rīgas rajons,Rīga,3000.0,7000.0
4,Informāciju sistēmu testētājs/-a (Middle QA En...,https://www.cv.lv/lv/vacancy/984685/workingday...,WorkingDay Latvia,https://www.cv.lv/lv/search/employer/workingda...,13.04.2023,13.05.2023,Latvija,Rīgas rajons,Rīga,2700.0,3500.0
...,...,...,...,...,...,...,...,...,...,...,...
1181,Viešųjų pirkimų koordinatorius,https://www.cv.lt/valstybes-tarnyba/komunikaci...,Lietuvos kultūros taryba,Nan,09.05.2023,Nan,Lithuania,Nan,Vilnius,NaN,NaN
1182,Resocializacijos skyriaus Vyriausiasis sociali...,https://www.cv.lt/valstybes-tarnyba/komunikaci...,Lietuvos kalėjimų tarnyba,Nan,09.05.2023,Nan,Lithuania,Nan,Vilnius,NaN,NaN
1183,Projektų įgyvendinimo ir priežiūros skyriaus v...,https://www.cv.lt/valstybes-tarnyba/komunikaci...,Lietuvos Respublikos aplinkos ministerijos Apl...,Nan,11.05.2023,Nan,Lithuania,Nan,Vilnius,0.0,0.0
1184,7-osios personalo administravimo grupės Person...,https://www.cv.lt/valstybes-tarnyba/komunikaci...,Nacionalinis bendrųjų funkcijų centras,Nan,10.05.2023,Nan,Lithuania,Nan,Other,NaN,NaN


### 4. Data Filtering

In [35]:
filtered_df = full_df[full_df['Title'].str.contains('anal|dat', case=False)]

In [36]:
filtered_df

,Title,Vacancy URL,Company name,Company URL,Published,End_date,Country,Region,City,Salary Range min,Salary Range max
10,Middle/Senior Data Engineer,https://www.cv.lv/lv/vacancy/985446/it-camp/mi...,IT Camp,https://www.cv.lv/lv/search/employer/it-camp?e...,14.04.2023,14.05.2023,Latvija,Rīgas rajons,Rīga,2900.0,5000.0
15,Data Protection Officer,https://www.cv.lv/lv/vacancy/985792/via-sms-gr...,VIA SMS Group,https://www.cv.lv/lv/search/employer/via-sms-g...,14.04.2023,14.05.2023,Latvija,Rīgas rajons,Rīga,2000.0,3000.0
20,HR DATU APSTRĀDES SPECIĀLISTS/-E AR VĀCU VALOD...,https://www.cv.lv/lv/vacancy/985581/cv-online-...,CV-Online Recruitment,https://www.cv.lv/lv/search/employer/cv-online...,13.05.2023,14.05.2023,Latvija,Rīgas rajons,Rīga,1600.0,NaN
31,Data Engineer,https://www.cv.lv/lv/vacancy/991049/luminor-ba...,Luminor Bank AS Latvijas filiāle,https://www.cv.lv/lv/search/employer/luminor-b...,25.04.2023,14.05.2023,Latvija,Rīgas rajons,Rīga,1850.0,3400.0
33,DATU PĀRVALDĪBAS UN PROJEKTU KOORDINATORS,https://www.cv.lv/lv/vacancy/988313/ergo-life-...,ERGO Life Insurance SE Latvijas filiāle,https://www.cv.lv/lv/search/employer/ergo-life...,11.05.2023,14.05.2023,Latvija,Rīgas rajons,Rīga,1200.0,1500.0
...,...,...,...,...,...,...,...,...,...,...,...
1119,Business Analyst in Capital Exposure Management,https://www.cv.lt/administravimo-darbai/admini...,Danske Bank A/S Lietuvos filialas,Nan,08.05.2023,Nan,Lithuania,Nan,Vilnius,2680.0,4020.0
1120,Business Analyst in Capital Exposure Management,https://www.cv.lt/administravimo-darbai/admini...,Danske Bank A/S Lietuvos filialas,Nan,05.05.2023,Nan,Lithuania,Nan,Vilnius,2680.0,4020.0
1160,Viešųjų finansų sistemos valdymo grupės finans...,https://www.cv.lt/teises-darbai/jaunesnysis-mo...,Lietuvos Respublikos finansų ministerija,Nan,11.05.2023,Nan,Lithuania,Nan,Vilnius,NaN,NaN
1168,Ekonominės analizės skyriaus vyriausiasis spec...,https://www.cv.lt/valstybes-tarnyba/komunikaci...,Valstybinė energetikos reguliavimo taryba,Nan,04.05.2023,Nan,Lithuania,Nan,Other,NaN,NaN


### 5. SQLite Database

In [37]:
import sqlite3
import pandas as pd

# create a connection to the database
conn = sqlite3.connect('Job_postings_CV.db')

# save the DataFrame to the database
filtered_df.to_sql('alldata', conn)

# close the connection
conn.close()

C:\Users\ernes\anaconda3\lib\site-packages\pandas\core\generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(
